In [7]:
import os
main_dir_names = [str(i) for i in range(12)]
def main():
    # Create directory
    for i in range(0, len(main_dir_names)):
        dirName=str(main_dir_names[i])
        try:
            # Create target Directory
            os.makedirs(dirName+'/')
            
            os.makedirs(dirName)

            print("Directory " , dirName ,  " Created ") 
        except FileExistsError:
            print("Directory " , dirName ,  " already exists")        
#         # Create target Directory if don't exist
#         if not os.path.exists(dirName):
#             os.makedirs(dirName)
#             print("Directory " , dirName ,  " Created ")
#         else:    
#             print("Directory " , dirName ,  " already exists")
if __name__ == '__main__':
    main()            

Directory  0  already exists
Directory  1  already exists
Directory  2  already exists
Directory  3  already exists
Directory  4  already exists
Directory  5  already exists
Directory  6  already exists
Directory  7  already exists
Directory  8  already exists
Directory  9  already exists
Directory  10  already exists
Directory  11  already exists


In [9]:
import h5py, numpy
from scipy.linalg import block_diag


tol = 1.e-8
Uarr=numpy.arange(12)
# read data in spin-faster basis
for U in Uarr:
    with h5py.File(f'../example/twobanddegeneratehubbardmodel/ed/hybr/'+"{:0.02f}".format(U)+"/HEmbed.h5", 'r') as f:
        d = f['impurity_0/D'][::2, ::2].T#[n:, n:]
        h1e = f['impurity_0/H1E'][::2, ::2].T#[n:, n:]
        lam = f['impurity_0/LAMBDA'][::2, ::2].T#[n:, n:]
        v2e = f['impurity_0/V2E'][::2, ::2, ::2, ::2].T
    #v2e = v2e[n:, n:, n:, n:]

    # one spin-block is needed.
    norb = h1e.shape[0]
    v1e = block_diag(h1e, -lam)
    v1e[:norb, norb:] = d.T
    v1e[norb:, :norb] = d.conj()
    with open(str(U)+"/hybr/v1e.dat", "w") as f:
        f.write(f"# norb: {norb} spin index NOT included \n")
        for i, vi in enumerate(v1e):
            for j, vij in enumerate(vi):
                if abs(vij) > tol:
                    f.write(f"{i:2d} {j:2d} {vij:15.9f}\n")

    with open(str(U)+"/hybr/v2e.dat", "w") as f:
        f.write(f"# norb: {norb} spin index NOT included \n")
        f.write(f"# chemsit's convention: v[i,j,k,l] c_i^\dag c_k^\dag c_l c_j\n")
        for i, vi in enumerate(v2e):
            for j, vij in enumerate(vi):
                for k, vijk in enumerate(vij):
                    for l, vijkl in enumerate(vijk):
                        if abs(vijkl) > tol:
                            f.write(f"{i:2d} {j:2d} {k:2d} {l:2d} {vijkl:15.9f}\n")


# with open("hembed.inp", "w") as f:
#     # NF
#     f.write(f"{norb*4}\n")
#     # Ne
#     f.write(f"{norb*2}\n")
#     # one-body
#     for i, vi in enumerate(v1e):
#         for j, vij in enumerate(vi):
#             f.write(f"{vij:15.9f}\n")
#             if i == j:
#                 break
#     # two-body
#     v2e2 = numpy.zeros([norb*2]*4)
#     v2e2[:norb, :norb, :norb, :norb] = v2e
#     for i, vi in enumerate(v2e2):
#         for j, vij in enumerate(vi):
#             for k, vijk in enumerate(vij):
#                 for l, vijkl in enumerate(vijk):
#                     f.write(f"{vijkl:15.9f}\n")
#             if i == j:
#                 break
